In [1]:
import os

In [2]:
%pwd # shows from which folder or directory u r using this application or files.

'd:\\Datascience_end_to_end_2\\end_to_end_machine_learn_2\\research'

In [3]:
os.chdir('../') #As we are in the research we have to go to the previous folder hence we use this funtion to do that ".."means goes one step back

In [4]:
%pwd

'd:\\Datascience_end_to_end_2\\end_to_end_machine_learn_2'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)

class DataIngestionConfig: # The class here used is from library dataclass hence we no need to use the self while declaring a variable/attribute.
    
    root_dir: Path
    source_URL: str
    local_data_file : Path
    unzip_dir:Path

In [6]:
from ml_project.constants import *
from ml_project.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [8]:
import os
import urllib.request as request
import zipfile
from ml_project import logger
from ml_project.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )

            logger.info(f"{filename} download! with following info: \n{headers}")

        else:
            logger.info(f"File already exists of size :{get_size(Path(self.config.local_data_file))}")

    
    def extract_zip_file(self):

        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns  None
        """

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok = True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)






In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config= data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()


except Exception as e:
    raise e


[2023-08-27 18:24:13,123: INFO: common: yaml file : config\config.yaml loaded successfully]
[2023-08-27 18:24:13,231: INFO: common: yaml file : params.yaml loaded successfully]
[2023-08-27 18:24:13,235: INFO: common: yaml file : schema.yaml loaded successfully]
[2023-08-27 18:24:13,235: INFO: common: created directory at : artifacts]
[2023-08-27 18:24:13,243: INFO: common: created directory at : artifacts/data_ingestion]
[2023-08-27 18:24:13,243: INFO: 3265247685: File already exists of size :~26 KB]
